<a href="https://colab.research.google.com/github/dejiandrew/nba-award-predictor/blob/deji/pow_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wget
import pandas as pd
import numpy as np
import duckdb
import wget
import os

# Download CSV files
wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/nba-all-stars.csv')
wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/nba-mvp.csv')
wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/all-nba-first-team.csv')
wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/all-nba-second-team.csv')
wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/all-nba-third-team.csv')
wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/player-of-the-week.csv')
wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/player-statistics.csv')
wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/games.csv')

'games (1).csv'

In [2]:
games_df = pd.read_csv('games.csv')
games_df

/tmp/ipython-input-3918661511.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  games_df = pd.read_csv('games.csv')


,gameId,gameDate,hometeamCity,hometeamName,hometeamId,awayteamCity,awayteamName,awayteamId,homeScore,awayScore,winner,gameType,attendance,arenaId,gameLabel,gameSubLabel,seriesGameNumber
0,22500239,2025-11-16T20:00:00Z,Phoenix,Suns,1610612756,Atlanta,Hawks,1610612737,122,124,1610612737,NaN,17071.0,NaN,NaN,NaN,NaN
1,22500240,2025-11-16T20:00:00Z,Utah,Jazz,1610612762,Chicago,Bulls,1610612741,150,147,1610612762,NaN,18186.0,NaN,NaN,NaN,NaN
2,22500238,2025-11-16T19:30:00Z,Dallas,Mavericks,1610612742,Portland,Trail Blazers,1610612757,138,133,1610612742,NaN,18808.0,NaN,NaN,NaN,NaN
3,22500236,2025-11-16T19:00:00Z,Houston,Rockets,1610612745,Orlando,Magic,1610612753,117,113,1610612745,NaN,18055.0,NaN,NaN,NaN,NaN
4,22500237,2025-11-16T19:00:00Z,New Orleans,Pelicans,1610612740,Golden State,Warriors,1610612744,106,124,1610612744,NaN,18373.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72144,24600083,1946-12-08 19:00:00,New York,Knicks,1610612752,Boston,Celtics,1610612738,62,44,1610612752,Regular Season,NaN,0.0,NaN,NaN,NaN
72145,24600078,1946-12-07 19:00:00,Boston,Celtics,1610612738,New York,Knicks,1610612752,65,90,1610612752,Regular Season,NaN,0.0,NaN,NaN,NaN
72146,24600076,1946-12-05 19:00:00,Philadelphia,Warriors,1610612744,New York,Knicks,1610612752,62,51,1610612744,Regular Season,NaN,0.0,NaN,NaN,NaN
72147,24600063,1946-11-30 19:00:00,New York,Knicks,1610612752,Philadelphia,Warriors,1610612744,64,60,1610612752,Regular Season,NaN,0.0,NaN,NaN,NaN


In [3]:
def build_team_games(df, filter=None):
    df = df.copy()

    # Parse datetime with mixed formats
    df['gamedate'] = pd.to_datetime(
        df['gamedate'],
        utc=True,
        errors='coerce',
        format='mixed',
        dayfirst=True,
    )

    if filter:
        df = df.query(filter)

    # Normalize: make two rows per game (home + away)
    home = df.rename(columns={
        'hometeamname': 'team',
        'hometeamid':   'teamid',
        'homescore':    'team_score',
        'awayscore':    'opp_score'
    })[['gameid','gamedate','team','teamid','team_score','opp_score','winner',
        'awayteamname','awayteamid']].assign(
            home = 1,
            opponent  = lambda x: x['awayteamname'],
            opponentid= lambda x: x['awayteamid'],
        ).drop(columns=['awayteamname','awayteamid'])

    away = df.rename(columns={
        'awayteamname': 'team',
        'awayteamid':   'teamid',
        'awayscore':    'team_score',
        'homescore':    'opp_score'
    })[['gameid','gamedate','team','teamid','team_score','opp_score','winner',
        'hometeamname','hometeamid']].assign(
            home = 0,
            opponent  = lambda x: x['hometeamname'],
            opponentid= lambda x: x['hometeamid'],
        ).drop(columns=['hometeamname','hometeamid'])

    long = pd.concat([home, away], ignore_index=True)

    # Outcome flags
    long['is_win']  = (long['teamid'] == long['winner']).astype(int)
    long['outcome'] = long['is_win'].map({1: 'win', 0: 'loss'})
    long['is_home_win'] = ((long['home'] == 1) & (long['is_win'] == 1)).astype(int)
    long['is_away_win'] = ((long['home'] == 0) & (long['is_win'] == 1)).astype(int)

    # Season (season starts in July)
    year = long['gamedate'].dt.year
    month = long['gamedate'].dt.month
    long['season'] = np.where(month >= 7, year, year - 1)

    # Sort for rolling calculations
    long = long.sort_values(['teamid', 'season', 'gamedate', 'gameid'], kind='mergesort')

    # Group by team + season for all season-based stats
    g = long.groupby(['teamid', 'season'], group_keys=False)

    # Games played prior (per season)
    long['games_prior'] = g.cumcount()

    # Wins / losses prior (per season)
    long['wins_prior'] = g['is_win'].transform(
        lambda s: s.shift().fillna(0).cumsum().astype(int)
    )
    long['losses_prior'] = long['games_prior'] - long['wins_prior']

    # Home / away games prior (per season)
    long['home_games_prior'] = g['home'].transform(
        lambda s: s.shift(fill_value=0).cumsum()
    )
    long['away_games_prior'] = long['games_prior'] - long['home_games_prior']

    # Home / away wins prior (per season)
    long['home_wins_prior'] = g['is_home_win'].transform(
        lambda s: s.shift().fillna(0).cumsum().astype(int)
    )
    long['away_wins_prior'] = g['is_away_win'].transform(
        lambda s: s.shift().fillna(0).cumsum().astype(int)
    )

    # Home / away losses prior (per season)
    long['home_losses_prior'] = long['home_games_prior'] - long['home_wins_prior']
    long['away_losses_prior'] = long['away_games_prior'] - long['away_wins_prior']

    # Win streaks prior (per season)
    def streak_prior(series):
        prior = series.shift().fillna(0).astype(int)
        # reset when we see a 0
        return prior.groupby((prior == 0).cumsum()).cumsum().astype(int)

    long['win_streak_prior'] = g['is_win'].transform(streak_prior)
    long['home_win_streak_prior'] = g['is_home_win'].transform(streak_prior)
    long['away_win_streak_prior'] = g['is_away_win'].transform(streak_prior)

    # Record strings (per season)
    long['record_prior'] = long['wins_prior'].astype(str) + '-' + long['losses_prior'].astype(str)
    long['home_record_prior'] = long['home_wins_prior'].astype(str) + '-' + long['home_losses_prior'].astype(str)
    long['away_record_prior'] = long['away_wins_prior'].astype(str) + '-' + long['away_losses_prior'].astype(str)

    # Opponent's prior record for same game (still season-based because wins_prior/losses_prior are)
    opp_prior = long[['gameid','teamid','wins_prior','losses_prior']].rename(
        columns={
            'teamid': 'opponentid',
            'wins_prior': 'opp_wins_prior',
            'losses_prior': 'opp_losses_prior'
        }
    )

    long = long.merge(opp_prior, on=['gameid','opponentid'], how='left')

    # Opponent winrate prior to the game
    opp_games_prior = long['opp_wins_prior'] + long['opp_losses_prior']
    long['opp_winrate_prior'] = long['opp_wins_prior'] / opp_games_prior.where(opp_games_prior > 0)

    # Flag for wins vs > .500 opponent (per game)
    long['is_win_vs_over_500'] = (
        (long['is_win'] == 1) &
        (long['opp_winrate_prior'] > 0.5)
    ).astype(int)

    # Per-team, per-season cumulative PRIOR wins vs > .500
    long['wins_vs_over_500_prior'] = (
        long.groupby(['teamid', 'season'])['is_win_vs_over_500']
            .transform(lambda s: s.shift().fillna(0).cumsum().astype(int))
    )

    # Week-based (Mon–Sun) record PRIOR, per-season
    iso = long['gamedate'].dt.isocalendar()
    long['week_year'] = iso['year']
    long['week_num']  = iso['week']

    gw = long.groupby(['teamid','season','week_year','week_num'], group_keys=False)

    long['week_games_prior'] = gw.cumcount()
    long['week_wins_prior'] = gw['is_win'].transform(
        lambda s: s.shift().fillna(0).cumsum().astype(int)
    )
    long['week_losses_prior'] = long['week_games_prior'] - long['week_wins_prior']

    long['week_record_prior'] = (
        long['week_wins_prior'].astype(str)
        + '-' +
        long['week_losses_prior'].astype(str)
    )

    # Final output
    out = long[['gameid','gamedate',
                'team','teamid',
                'opponent','opponentid',
                'outcome','home',
                'team_score','opp_score',
                'games_prior','wins_prior','losses_prior','record_prior',
                'home_games_prior','home_wins_prior','home_losses_prior','home_record_prior',
                'away_games_prior','away_wins_prior','away_losses_prior','away_record_prior',
                'win_streak_prior','home_win_streak_prior','away_win_streak_prior',
                'opp_wins_prior','opp_losses_prior','opp_winrate_prior',
                'is_win_vs_over_500','wins_vs_over_500_prior',
                'week_games_prior','week_wins_prior','week_losses_prior','week_record_prior',
                'season'
               ]]
    out.loc[:, 'gameid'] = out['gameid'].astype(int)
    out = out.rename(columns={'gameid': 'gameId'})

    return out.sort_values(['gamedate','gameId','home']).reset_index(drop=True)

In [4]:
games_df.columns = games_df.columns.str.lower()

In [5]:
game = build_team_games(games_df, "gamedate.dt.year >= 1979")
game

,gameId,gamedate,team,teamid,opponent,opponentid,outcome,home,team_score,opp_score,...,opp_wins_prior,opp_losses_prior,opp_winrate_prior,is_win_vs_over_500,wins_vs_over_500_prior,week_games_prior,week_wins_prior,week_losses_prior,week_record_prior,season
0,27800397,1979-01-01 20:00:00+00:00,Warriors,1610612744,SuperSonics,1610612760,win,0,110,97,...,0,0,NaN,0,0,0,0,0,0-0,1978
1,27800397,1979-01-01 20:00:00+00:00,SuperSonics,1610612760,Warriors,1610612744,loss,1,97,110,...,0,0,NaN,0,0,0,0,0,0-0,1978
2,27800398,1979-01-02 20:00:00+00:00,Nuggets,1610612743,Kings,1610612758,win,0,99,97,...,0,0,NaN,0,0,0,0,0,0-0,1978
3,27800398,1979-01-02 20:00:00+00:00,Kings,1610612758,Nuggets,1610612743,loss,1,97,99,...,0,0,NaN,0,0,0,0,0,0-0,1978
4,27800399,1979-01-02 20:00:00+00:00,Bulls,1610612741,Bullets,1610612764,loss,0,86,109,...,0,0,NaN,0,0,0,0,0,0-0,1978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115513,22500238,2025-11-16 19:30:00+00:00,Mavericks,1610612742,Trail Blazers,1610612757,win,1,138,133,...,7,9,0.437500,0,2,3,0,3,0-3,2025
115514,22500239,2025-11-16 20:00:00+00:00,Hawks,1610612737,Suns,1610612756,win,0,124,122,...,11,6,0.647059,1,3,3,3,0,3-0,2025
115515,22500239,2025-11-16 20:00:00+00:00,Suns,1610612756,Hawks,1610612737,loss,1,122,124,...,10,7,0.588235,0,3,3,3,0,3-0,2025
115516,22500240,2025-11-16 20:00:00+00:00,Bulls,1610612741,Jazz,1610612762,loss,0,147,150,...,5,11,0.312500,0,3,2,0,2,0-2,2025


In [6]:
game.isnull().sum()

,0
gameId,0
gamedate,0
team,0
teamid,0
opponent,0
opponentid,0
outcome,0
home,0
team_score,0
opp_score,0


In [7]:
query = """
WITH CTE AS (
SELECT
season
,team
,MIN(CAST(gamedate AS DATE)) AS min_gamedate
,MAX(CAST(gamedate AS DATE)) AS max_gamedate
FROM game
WHERE opp_winrate_prior IS NULL
GROUP BY season, team
)

SELECT *
,MONTH(min_gamedate) AS min_gamedate_month
,MONTH(max_gamedate) AS max_gamedate_month
FROM CTE
WHERE 1=1
AND min_gamedate_month NOT IN (9,10,11)           --This removes Sept,Oct,Nov games (1st game of season)
AND (season != 2020 AND min_gamedate_month != 7)  --Remove 2020 NBA Bubble scrimmages
AND (season != 1999 AND min_gamedate_month != 1)  --Remove 1999 lockout
AND (season != 1999 AND min_gamedate_month != 2)  --Remove 1999 lockout
AND (season != 2011 AND min_gamedate_month != 12) --Remove 2011 lockout


"""

duckdb.query(query).df()

# Based on this analysis, the null columns in game["opp_winrate_prior"] represent games where
# the opponent had played zero regular season games at that point. We can set game["opp_winrate_prior"]
# to 0.500 for these rows.


,season,team,min_gamedate,max_gamedate,min_gamedate_month,max_gamedate_month


In [8]:
game["opp_winrate_prior"] = game["opp_winrate_prior"].fillna(0.500)

In [9]:
game

,gameId,gamedate,team,teamid,opponent,opponentid,outcome,home,team_score,opp_score,...,opp_wins_prior,opp_losses_prior,opp_winrate_prior,is_win_vs_over_500,wins_vs_over_500_prior,week_games_prior,week_wins_prior,week_losses_prior,week_record_prior,season
0,27800397,1979-01-01 20:00:00+00:00,Warriors,1610612744,SuperSonics,1610612760,win,0,110,97,...,0,0,0.500000,0,0,0,0,0,0-0,1978
1,27800397,1979-01-01 20:00:00+00:00,SuperSonics,1610612760,Warriors,1610612744,loss,1,97,110,...,0,0,0.500000,0,0,0,0,0,0-0,1978
2,27800398,1979-01-02 20:00:00+00:00,Nuggets,1610612743,Kings,1610612758,win,0,99,97,...,0,0,0.500000,0,0,0,0,0,0-0,1978
3,27800398,1979-01-02 20:00:00+00:00,Kings,1610612758,Nuggets,1610612743,loss,1,97,99,...,0,0,0.500000,0,0,0,0,0,0-0,1978
4,27800399,1979-01-02 20:00:00+00:00,Bulls,1610612741,Bullets,1610612764,loss,0,86,109,...,0,0,0.500000,0,0,0,0,0,0-0,1978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115513,22500238,2025-11-16 19:30:00+00:00,Mavericks,1610612742,Trail Blazers,1610612757,win,1,138,133,...,7,9,0.437500,0,2,3,0,3,0-3,2025
115514,22500239,2025-11-16 20:00:00+00:00,Hawks,1610612737,Suns,1610612756,win,0,124,122,...,11,6,0.647059,1,3,3,3,0,3-0,2025
115515,22500239,2025-11-16 20:00:00+00:00,Suns,1610612756,Hawks,1610612737,loss,1,122,124,...,10,7,0.588235,0,3,3,3,0,3-0,2025
115516,22500240,2025-11-16 20:00:00+00:00,Bulls,1610612741,Jazz,1610612762,loss,0,147,150,...,5,11,0.312500,0,3,2,0,2,0-2,2025


In [10]:
game.columns

Index(['gameId', 'gamedate', 'team', 'teamid', 'opponent', 'opponentid',
       'outcome', 'home', 'team_score', 'opp_score', 'games_prior',
       'wins_prior', 'losses_prior', 'record_prior', 'home_games_prior',
       'home_wins_prior', 'home_losses_prior', 'home_record_prior',
       'away_games_prior', 'away_wins_prior', 'away_losses_prior',
       'away_record_prior', 'win_streak_prior', 'home_win_streak_prior',
       'away_win_streak_prior', 'opp_wins_prior', 'opp_losses_prior',
       'opp_winrate_prior', 'is_win_vs_over_500', 'wins_vs_over_500_prior',
       'week_games_prior', 'week_wins_prior', 'week_losses_prior',
       'week_record_prior', 'season'],
      dtype='object')

In [11]:
pow_df = pd.read_csv('player-of-the-week.csv')
pow_df.columns[pow_df.isna().any()]

Index(['conference'], dtype='object')

In [12]:
query = """
WITH CTE AS (
SELECT
player_id
,season
,player
,CASE
    -- Eastern Conference
    WHEN team = 'Boston Celtics' THEN 'East'
    WHEN team = 'Brooklyn Nets' THEN 'East'
    WHEN team = 'New York Knicks' THEN 'East'
    WHEN team = 'Philadelphia 76ers' THEN 'East'
    WHEN team = 'Philadelphia Sixers' THEN 'East'
    WHEN team = 'Toronto Raptors' THEN 'East'
    WHEN team = 'Chicago Bulls' THEN 'East'
    WHEN team = 'Cleveland Cavaliers' THEN 'East'
    WHEN team = 'Detroit Pistons' THEN 'East'
    WHEN team = 'Indiana Pacers' THEN 'East'
    WHEN team = 'Milwaukee Bucks' THEN 'East'
    WHEN team = 'Atlanta Hawks' THEN 'East'
    WHEN team = 'Charlotte Hornets' THEN 'East'
    WHEN team = 'Miami Heat' THEN 'East'
    WHEN team = 'Orlando Magic' THEN 'East'
    WHEN team = 'Washington Wizards' THEN 'East'
    WHEN team = 'Washington Bullets' THEN 'East'
    WHEN team = 'New Jersey Nets' THEN 'East'
    WHEN team = 'Charlotte Bobcats' THEN 'East'

    -- Western Conference
    WHEN team = 'Denver Nuggets' THEN 'West'
    WHEN team = 'Minnesota Timberwolves' THEN 'West'
    WHEN team = 'Oklahoma City Thunder' THEN 'West'
    WHEN team = 'Portland Trail Blazers' THEN 'West'
    WHEN team = 'Utah Jazz' THEN 'West'
    WHEN team = 'Golden State Warriors' THEN 'West'
    WHEN team = 'Los Angeles Clippers' THEN 'West'
    WHEN team = 'Los Angeles Lakers' THEN 'West'
    WHEN team = 'Phoenix Suns' THEN 'West'
    WHEN team = 'Sacramento Kings' THEN 'West'
    WHEN team = 'Dallas Mavericks' THEN 'West'
    WHEN team = 'Houston Rockets' THEN 'West'
    WHEN team = 'Memphis Grizzlies' THEN 'West'
    WHEN team = 'New Orleans Pelicans' THEN 'West'
    WHEN team = 'San Antonio Spurs' THEN 'West'
    WHEN team = 'Seattle SuperSonics' THEN 'West'
    WHEN team = 'San Diego Clippers' THEN 'West'
    WHEN team = 'Kansas City Kings' THEN 'West'
    WHEN team = 'New Orleans Hornets' THEN 'West'
END AS conference

,date
,DAY(CAST(date AS DATE)) AS day
,WEEK(CAST(date AS DATE)) AS week
,MONTH(CAST(date AS DATE)) AS month
,YEAR(CAST(date AS DATE)) AS year
,team
,pos
,height
,weight
,age
,"Pre-Draft Team"
,"Draft Yr"
,yos

FROM pow_df
)

SELECT * FROM CTE

"""

pow_df = duckdb.query(query).df()
pow_df

,player_id,season,player,conference,date,day,week,month,year,team,pos,height,weight,age,Pre-Draft Team,Draft Yr,yos
0,1630595,2025-2026,Cade Cunningham,East,2025-11-10,10,46,11,2025,Detroit Pistons,SF,6-6,220,24,Oklahoma State,2021,4
1,203999,2025-2026,Nikola Jokic,West,2025-11-10,10,46,11,2025,Denver Nuggets,C,6-11,284,31,KK Mega Bemax (Serbia),2014,10
2,1630178,2025-2026,Tyrese Maxey,East,2025-11-03,3,45,11,2025,Philadelphia Sixers,PG,6-2,200,25,Kentucky,2020,5
3,1628983,2025-2026,Shai Gilgeous-Alexander,West,2025-11-03,3,45,11,2025,Oklahoma City Thunder,PG,6-6,195,27,Kentucky,2018,7
4,203507,2025-2026,Giannis Antetokounmpo,East,2025-10-27,27,44,10,2025,Milwaukee Bucks,F,6-11,243,31,Filathlitikos Div II Greece (Greece),2013,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1561,76736,1979-1980,Phil Ford,West,1979-11-18,18,46,11,1979,Kansas City Kings,G,6-2,175,24,North Carolina,1978,1
1562,77142,1979-1980,Magic Johnson,West,1979-11-11,11,45,11,1979,Los Angeles Lakers,PG,6-9,255,20,Michigan State,1979,0
1563,77160,1979-1980,Marques Johnson,East,1979-11-04,4,44,11,1979,Milwaukee Bucks,GF,6-7,218,24,UCLA,1977,2
1564,77952,1979-1980,Micheal Ray Richardson,East,1979-10-28,28,43,10,1979,New York Knicks,PG,6-5,189,24,Montana,1978,1


In [13]:
game.columns

Index(['gameId', 'gamedate', 'team', 'teamid', 'opponent', 'opponentid',
       'outcome', 'home', 'team_score', 'opp_score', 'games_prior',
       'wins_prior', 'losses_prior', 'record_prior', 'home_games_prior',
       'home_wins_prior', 'home_losses_prior', 'home_record_prior',
       'away_games_prior', 'away_wins_prior', 'away_losses_prior',
       'away_record_prior', 'win_streak_prior', 'home_win_streak_prior',
       'away_win_streak_prior', 'opp_wins_prior', 'opp_losses_prior',
       'opp_winrate_prior', 'is_win_vs_over_500', 'wins_vs_over_500_prior',
       'week_games_prior', 'week_wins_prior', 'week_losses_prior',
       'week_record_prior', 'season'],
      dtype='object')

In [14]:
query = """
SELECT
gameId
,CAST(gamedate AS DATE) AS gamedate
,DAY(CAST(gamedate AS DATE)) AS day
,WEEK(CAST(gamedate AS DATE)) AS week
,MONTH(CAST(gamedate AS DATE)) AS month
,YEAR(CAST(gamedate AS DATE)) AS year
,team
,teamid
,opponent
,opponentid
,outcome
,home
,team_score
,opp_score
,games_prior
,wins_prior
,losses_prior
,record_prior
,home_games_prior
,home_wins_prior
,home_losses_prior
,home_record_prior
,away_games_prior
,away_wins_prior
,away_losses_prior
,away_record_prior
,win_streak_prior
,home_win_streak_prior
,away_win_streak_prior
,opp_wins_prior
,opp_losses_prior
,opp_winrate_prior
,is_win_vs_over_500
,wins_vs_over_500_prior
,week_games_prior
,week_wins_prior
,week_losses_prior
,week_record_prior
,season

FROM game

"""

game = duckdb.query(query).df()
game

,gameId,gamedate,day,week,month,year,team,teamid,opponent,opponentid,...,opp_wins_prior,opp_losses_prior,opp_winrate_prior,is_win_vs_over_500,wins_vs_over_500_prior,week_games_prior,week_wins_prior,week_losses_prior,week_record_prior,season
0,27800397,1979-01-01,1,1,1,1979,Warriors,1610612744,SuperSonics,1610612760,...,0,0,0.500000,0,0,0,0,0,0-0,1978
1,27800397,1979-01-01,1,1,1,1979,SuperSonics,1610612760,Warriors,1610612744,...,0,0,0.500000,0,0,0,0,0,0-0,1978
2,27800398,1979-01-02,2,1,1,1979,Nuggets,1610612743,Kings,1610612758,...,0,0,0.500000,0,0,0,0,0,0-0,1978
3,27800398,1979-01-02,2,1,1,1979,Kings,1610612758,Nuggets,1610612743,...,0,0,0.500000,0,0,0,0,0,0-0,1978
4,27800399,1979-01-02,2,1,1,1979,Bulls,1610612741,Bullets,1610612764,...,0,0,0.500000,0,0,0,0,0,0-0,1978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115513,22500238,2025-11-16,16,46,11,2025,Mavericks,1610612742,Trail Blazers,1610612757,...,7,9,0.437500,0,2,3,0,3,0-3,2025
115514,22500239,2025-11-16,16,46,11,2025,Hawks,1610612737,Suns,1610612756,...,11,6,0.647059,1,3,3,3,0,3-0,2025
115515,22500239,2025-11-16,16,46,11,2025,Suns,1610612756,Hawks,1610612737,...,10,7,0.588235,0,3,3,3,0,3-0,2025
115516,22500240,2025-11-16,16,46,11,2025,Bulls,1610612741,Jazz,1610612762,...,5,11,0.312500,0,3,2,0,2,0-2,2025


In [85]:
query = """
WITH CTE AS (
SELECT *
FROM game
LEFT JOIN pow_df
ON game.month = pow_df.month AND game.week = pow_df.week AND game.year = pow_df.year
)

SELECT *,
CASE WHEN yos IS NULL THEN 0 ELSE 1 END AS won_player_of_the_week
FROM CTE
"""

combined_df = duckdb.query(query).df()
# combined_df = combined_df[['gameId', 'gamedate', 'team', 'teamid', 'opponent', 'opponentid',
#        'outcome', 'home', 'team_score', 'opp_score', 'games_prior',
#        'wins_prior', 'losses_prior', 'record_prior', 'home_games_prior',
#        'home_wins_prior', 'home_losses_prior', 'home_record_prior',
#        'away_games_prior', 'away_wins_prior', 'away_losses_prior',
#        'away_record_prior', 'win_streak_prior', 'home_win_streak_prior',
#        'away_win_streak_prior', 'opp_wins_prior', 'opp_losses_prior',
#        'opp_winrate_prior', 'is_win_vs_over_500', 'wins_vs_over_500_prior',
#        'week_games_prior', 'week_wins_prior', 'week_losses_prior',
#        'week_record_prior', 'season', 'date', 'year', 'month', 'week', 'day',
#        'conference', 'player', 'player_id', 'age', 'height', 'pos', 'yos']]

combined_df

,gameId,gamedate,day,week,month,year,team,teamid,opponent,opponentid,...,year_1,team_1,pos,height,weight,age,Pre-Draft Team,Draft Yr,yos,won_player_of_the_week
0,22000447,2021-02-18,18,7,2,2021,Heat,1610612748,Kings,1610612758,...,2021,Phoenix Suns,SG,6-5,206,24,Kentucky,2015,5,1
1,22000447,2021-02-18,18,7,2,2021,Kings,1610612758,Heat,1610612748,...,2021,Phoenix Suns,SG,6-5,206,24,Kentucky,2015,5,1
2,22000449,2021-02-19,19,7,2,2021,Warriors,1610612744,Magic,1610612753,...,2021,Phoenix Suns,SG,6-5,206,24,Kentucky,2015,5,1
3,22000449,2021-02-19,19,7,2,2021,Magic,1610612753,Warriors,1610612744,...,2021,Phoenix Suns,SG,6-5,206,24,Kentucky,2015,5,1
4,22000563,2021-02-19,19,7,2,2021,Nuggets,1610612743,Cavaliers,1610612739,...,2021,Phoenix Suns,SG,6-5,206,24,Kentucky,2015,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163933,22000090,2021-01-03,3,53,1,2021,Bulls,1610612741,Mavericks,1610612742,...,<NA>,None,None,None,<NA>,<NA>,None,<NA>,<NA>,0
163934,22000091,2021-01-03,3,53,1,2021,Clippers,1610612746,Suns,1610612756,...,<NA>,None,None,None,<NA>,<NA>,None,<NA>,<NA>,0
163935,22000091,2021-01-03,3,53,1,2021,Suns,1610612756,Clippers,1610612746,...,<NA>,None,None,None,<NA>,<NA>,None,<NA>,<NA>,0
163936,22000092,2021-01-03,3,53,1,2021,Trail Blazers,1610612757,Warriors,1610612744,...,<NA>,None,None,None,<NA>,<NA>,None,<NA>,<NA>,0


In [16]:
game.columns

Index(['gameId', 'gamedate', 'day', 'week', 'month', 'year', 'team', 'teamid',
       'opponent', 'opponentid', 'outcome', 'home', 'team_score', 'opp_score',
       'games_prior', 'wins_prior', 'losses_prior', 'record_prior',
       'home_games_prior', 'home_wins_prior', 'home_losses_prior',
       'home_record_prior', 'away_games_prior', 'away_wins_prior',
       'away_losses_prior', 'away_record_prior', 'win_streak_prior',
       'home_win_streak_prior', 'away_win_streak_prior', 'opp_wins_prior',
       'opp_losses_prior', 'opp_winrate_prior', 'is_win_vs_over_500',
       'wins_vs_over_500_prior', 'week_games_prior', 'week_wins_prior',
       'week_losses_prior', 'week_record_prior', 'season'],
      dtype='object')

In [87]:
player_statistics_df = pd.read_csv('player-statistics.csv')
player_statistics_df.head()

/tmp/ipython-input-1148877721.py:1: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  player_statistics_df = pd.read_csv('player-statistics.csv')


,firstName,lastName,full_name,player_id,gameId,gameDate,playerteamCity,playerteamName,opponentteamCity,opponentteamName,...,threePointersPercentage,freeThrowsAttempted,freeThrowsMade,freeThrowsPercentage,reboundsDefensive,reboundsOffensive,reboundsTotal,foulsPersonal,turnovers,plusMinusPoints
0,Devin,Booker,Devin Booker,1626164,22500239,2025-11-16T20:00:00Z,Phoenix,Suns,Atlanta,Hawks,...,0.000,7.0,7.0,1.000,1.0,1.0,2.0,4.0,5.0,-17.0
1,Royce,O'Neale,Royce O'Neale,1626220,22500239,2025-11-16T20:00:00Z,Phoenix,Suns,Atlanta,Hawks,...,0.167,0.0,0.0,0.000,4.0,2.0,6.0,1.0,0.0,-18.0
2,Lauri,Markkanen,Lauri Markkanen,1628374,22500240,2025-11-16T20:00:00Z,Utah,Jazz,Chicago,Bulls,...,0.353,13.0,11.0,0.846,5.0,2.0,7.0,3.0,1.0,4.0
3,Luke,Kennard,Luke Kennard,1628379,22500239,2025-11-16T20:00:00Z,Atlanta,Hawks,Phoenix,Suns,...,0.000,0.0,0.0,0.000,1.0,0.0,1.0,0.0,0.0,-4.0
4,Dillon,Brooks,Dillon Brooks,1628415,22500239,2025-11-16T20:00:00Z,Phoenix,Suns,Atlanta,Hawks,...,0.429,3.0,3.0,1.000,5.0,1.0,6.0,5.0,3.0,-2.0


In [18]:
player_statistics_df.columns
# player_statistics_df.merge(combined_df,on='player_id',how='inner')

Index(['firstName', 'lastName', 'full_name', 'player_id', 'gameId', 'gameDate',
       'playerteamCity', 'playerteamName', 'opponentteamCity',
       'opponentteamName', 'gameType', 'gameLabel', 'gameSubLabel',
       'seriesGameNumber', 'win', 'home', 'numMinutes', 'points', 'assists',
       'blocks', 'steals', 'fieldGoalsAttempted', 'fieldGoalsMade',
       'fieldGoalsPercentage', 'threePointersAttempted', 'threePointersMade',
       'threePointersPercentage', 'freeThrowsAttempted', 'freeThrowsMade',
       'freeThrowsPercentage', 'reboundsDefensive', 'reboundsOffensive',
       'reboundsTotal', 'foulsPersonal', 'turnovers', 'plusMinusPoints'],
      dtype='object')

In [88]:
player_statistics_df = player_statistics_df[['firstName', 'lastName', 'full_name', 'player_id', 'gameId', 'gameDate',
  'numMinutes', 'points', 'assists',
       'blocks', 'steals', 'fieldGoalsAttempted', 'fieldGoalsMade',
       'fieldGoalsPercentage', 'threePointersAttempted', 'threePointersMade',
       'threePointersPercentage', 'freeThrowsAttempted', 'freeThrowsMade',
       'freeThrowsPercentage', 'reboundsDefensive', 'reboundsOffensive',
       'reboundsTotal', 'foulsPersonal', 'turnovers', 'plusMinusPoints']]
player_statistics_df

,firstName,lastName,full_name,player_id,gameId,gameDate,numMinutes,points,assists,blocks,...,threePointersPercentage,freeThrowsAttempted,freeThrowsMade,freeThrowsPercentage,reboundsDefensive,reboundsOffensive,reboundsTotal,foulsPersonal,turnovers,plusMinusPoints
0,Devin,Booker,Devin Booker,1626164,22500239,2025-11-16T20:00:00Z,37.36,27.0,7.0,1.0,...,0.000,7.0,7.0,1.000,1.0,1.0,2.0,4.0,5.0,-17.0
1,Royce,O'Neale,Royce O'Neale,1626220,22500239,2025-11-16T20:00:00Z,24.56,3.0,1.0,0.0,...,0.167,0.0,0.0,0.000,4.0,2.0,6.0,1.0,0.0,-18.0
2,Lauri,Markkanen,Lauri Markkanen,1628374,22500240,2025-11-16T20:00:00Z,46.22,47.0,2.0,2.0,...,0.353,13.0,11.0,0.846,5.0,2.0,7.0,3.0,1.0,4.0
3,Luke,Kennard,Luke Kennard,1628379,22500239,2025-11-16T20:00:00Z,11.35,0.0,0.0,0.0,...,0.000,0.0,0.0,0.000,1.0,0.0,1.0,0.0,0.0,-4.0
4,Dillon,Brooks,Dillon Brooks,1628415,22500239,2025-11-16T20:00:00Z,35.32,34.0,1.0,0.0,...,0.429,3.0,3.0,1.000,5.0,1.0,6.0,5.0,3.0,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1635253,Fred,Sheffield,Fred Sheffield,78131,24600052,1946-11-26 19:00:00,0.00,1.0,0.0,0.0,...,0.000,0.0,1.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
1635254,Connie,Simmons,Connie Simmons,78153,24600052,1946-11-26 19:00:00,0.00,15.0,0.0,0.0,...,0.000,0.0,5.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
1635255,Johnny,Simmons,Johnny Simmons,78154,24600052,1946-11-26 19:00:00,0.00,14.0,0.0,0.0,...,0.000,0.0,2.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
1635256,Virgil,Vaughn,Virgil Vaughn,78411,24600052,1946-11-26 19:00:00,0.00,4.0,0.0,0.0,...,0.000,0.0,2.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0


In [90]:
query = """

SELECT
player_statistics_df.firstName
,player_statistics_df.lastName
,player_statistics_df.full_name
,player_statistics_df.player_id
,player_statistics_df.gameId
,CAST(player_statistics_df.gameDate AS DATE) AS gameDate
,DAY(CAST(player_statistics_df.gameDate AS DATE)) AS day
,MONTH(CAST(player_statistics_df.gameDate AS DATE)) AS month
,YEAR(CAST(player_statistics_df.gameDate AS DATE)) AS year
,numMinutes
,points
,assists
,blocks
,steals
,fieldGoalsAttempted
,fieldGoalsMade
,fieldGoalsPercentage
,threePointersAttempted
,threePointersMade
,threePointersPercentage
,freeThrowsAttempted
,freeThrowsMade
,freeThrowsPercentage
,reboundsDefensive
,reboundsOffensive
,reboundsTotal
,foulsPersonal
,turnovers
,plusMinusPoints
,combined_df.*

FROM player_statistics_df
LEFT JOIN combined_df
ON player_statistics_df.player_id = combined_df.player_id AND player_statistics_df.gameId = combined_df.gameId
WHERE 1=1
AND numMinutes IS NOT NULL  -- Remove any games where a player didn't play

"""

player_stats_with_pow = duckdb.query(query).df()
player_stats_with_pow["won_player_of_the_week"] = np.where(player_stats_with_pow["won_player_of_the_week"].isna(),0,1)
# player_stats_with_pow = player_stats_with_pow[[
#     'firstName', 'lastName', 'full_name', 'player_id', 'gameId', 'gameDate',
#     'numMinutes', 'points', 'assists',
#     'blocks', 'steals', 'fieldGoalsAttempted', 'fieldGoalsMade',
#     'fieldGoalsPercentage', 'threePointersAttempted', 'threePointersMade',
#     'threePointersPercentage', 'freeThrowsAttempted', 'freeThrowsMade',
#     'freeThrowsPercentage', 'reboundsDefensive', 'reboundsOffensive',
#     'reboundsTotal', 'foulsPersonal', 'turnovers', 'plusMinusPoints', 'won_player_of_the_week'
# ]]
player_stats_with_pow

,firstName,lastName,full_name,player_id,gameId,gameDate,day,month,year,numMinutes,...,year_1_1,team_1,pos,height,weight,age,Pre-Draft Team,Draft Yr,yos,won_player_of_the_week
0,Dwight,Howard,Dwight Howard,2730,20700142,2007-11-18,18,11,2007,41.0,...,2007,Orlando Magic,C,6-11,265,22,Southwest Atlanta Christian Academy (Georgia),2004,3,1
1,Dwight,Howard,Dwight Howard,2730,20700142,2007-11-18,18,11,2007,41.0,...,2007,Orlando Magic,C,6-11,265,22,Southwest Atlanta Christian Academy (Georgia),2004,3,1
2,LeBron,James,LeBron James,2544,20700151,2007-11-20,20,11,2007,44.0,...,2007,Cleveland Cavaliers,F,6-9,250,23,St. Vincent St. Mary High School (Ohio),2003,4,1
3,LeBron,James,LeBron James,2544,20700151,2007-11-20,20,11,2007,44.0,...,2007,Cleveland Cavaliers,F,6-9,250,23,St. Vincent St. Mary High School (Ohio),2003,4,1
4,LeBron,James,LeBron James,2544,20700165,2007-11-21,21,11,2007,39.0,...,2007,Cleveland Cavaliers,F,6-9,250,23,St. Vincent St. Mary High School (Ohio),2003,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1476968,Jerry,Sloan,Jerry Sloan,78173,26500250,1966-02-05,5,2,1966,0.0,...,<NA>,None,None,None,<NA>,<NA>,None,<NA>,<NA>,0
1476969,Dick,Van Arsdale,Dick Van Arsdale,78398,26500249,1966-02-05,5,2,1966,0.0,...,<NA>,None,None,None,<NA>,<NA>,None,<NA>,<NA>,0
1476970,Cliff,Hagan,Cliff Hagan,76912,26500248,1966-02-04,4,2,1966,21.0,...,<NA>,None,None,None,<NA>,<NA>,None,<NA>,<NA>,0
1476971,Don,Nelson,Don Nelson,77700,26500248,1966-02-04,4,2,1966,25.0,...,<NA>,None,None,None,<NA>,<NA>,None,<NA>,<NA>,0


You have to fill in the nulls by creating a separate lookup table to capture those features if a player didn't win POW


Now bring in Deji's code

In [38]:
player_statistics_test_df = pd.read_csv('player-statistics.csv')

/tmp/ipython-input-4088499775.py:1: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  player_statistics_test_df = pd.read_csv('player-statistics.csv')


In [39]:
query = """

SELECT *,
    CASE
        WHEN gameDate BETWEEN '2025-10-02' AND '2026-08-01' THEN 2025
        WHEN gameDate BETWEEN '2024-10-04' AND '2025-06-22' THEN 2024
        WHEN gameDate BETWEEN '2023-10-05' AND '2024-06-17' THEN 2023
        WHEN gameDate BETWEEN '2022-09-30' AND '2023-06-12' THEN 2022
        WHEN gameDate BETWEEN '2021-10-03' AND '2022-06-16' THEN 2021
        WHEN gameDate BETWEEN '2020-12-11' AND '2021-07-20' THEN 2020
        WHEN gameDate BETWEEN '2019-10-22' AND '2020-10-11' THEN 2019
        WHEN gameDate BETWEEN '2018-10-16' AND '2019-06-10' THEN 2018
        WHEN gameDate BETWEEN '2017-09-30' AND '2018-06-08' THEN 2017
        WHEN gameDate BETWEEN '2016-10-01' AND '2017-06-12' THEN 2016
        WHEN gameDate BETWEEN '2015-10-02' AND '2016-06-19' THEN 2015
        WHEN gameDate BETWEEN '2014-10-04' AND '2015-06-16' THEN 2014
        WHEN gameDate BETWEEN '2013-10-05' AND '2014-06-15' THEN 2013
        WHEN gameDate BETWEEN '2012-10-05' AND '2013-06-20' THEN 2012
        WHEN gameDate BETWEEN '2011-12-16' AND '2012-06-21' THEN 2011
        WHEN gameDate BETWEEN '2010-10-03' AND '2011-06-12' THEN 2010
        WHEN gameDate BETWEEN '2009-10-01' AND '2010-06-17' THEN 2009
        WHEN gameDate BETWEEN '2008-10-05' AND '2009-06-11' THEN 2008
        WHEN gameDate BETWEEN '2007-10-06' AND '2008-06-17' THEN 2007
        WHEN gameDate BETWEEN '2006-10-05' AND '2007-06-14' THEN 2006
        WHEN gameDate BETWEEN '2005-10-10' AND '2006-04-19' THEN 2005
        WHEN gameDate BETWEEN '2004-11-02' AND '2005-06-21' THEN 2004
        WHEN gameDate BETWEEN '2003-10-28' AND '2004-06-15' THEN 2003
        WHEN gameDate BETWEEN '2002-10-29' AND '2003-06-15' THEN 2002
        WHEN gameDate BETWEEN '2001-10-30' AND '2002-04-17' THEN 2001
        WHEN gameDate BETWEEN '2000-10-31' AND '2001-06-15' THEN 2000
        WHEN gameDate BETWEEN '1999-11-02' AND '2000-04-19' THEN 1999
        WHEN gameDate BETWEEN '1998-02-05' AND '1999-06-25' THEN 1998
        WHEN gameDate BETWEEN '1997-10-31' AND '1998-06-14' THEN 1997
        WHEN gameDate BETWEEN '1996-11-01' AND '1997-06-13' THEN 1996
        WHEN gameDate BETWEEN '1995-11-03' AND '1996-04-21' THEN 1995
        WHEN gameDate BETWEEN '1994-11-04' AND '1995-06-14' THEN 1994
        WHEN gameDate BETWEEN '1993-11-05' AND '1994-04-24' THEN 1993
        WHEN gameDate BETWEEN '1992-11-06' AND '1993-06-20' THEN 1992
        WHEN gameDate BETWEEN '1991-11-01' AND '1992-06-12' THEN 1991
        WHEN gameDate BETWEEN '1990-11-02' AND '1991-06-12' THEN 1990
        WHEN gameDate BETWEEN '1989-11-03' AND '1990-06-14' THEN 1989
        WHEN gameDate BETWEEN '1988-11-04' AND '1989-06-13' THEN 1988
        WHEN gameDate BETWEEN '1987-11-06' AND '1988-06-19' THEN 1987
        WHEN gameDate BETWEEN '1986-10-31' AND '1987-06-14' THEN 1986
        WHEN gameDate BETWEEN '1985-10-25' AND '1986-06-05' THEN 1985
        WHEN gameDate BETWEEN '1984-10-26' AND '1985-06-09' THEN 1984
        WHEN gameDate BETWEEN '1983-10-28' AND '1984-06-12' THEN 1983
        WHEN gameDate BETWEEN '1982-10-29' AND '1983-05-31' THEN 1982
        WHEN gameDate BETWEEN '1981-10-30' AND '1982-06-06' THEN 1981
        WHEN gameDate BETWEEN '1980-10-10' AND '1981-05-14' THEN 1980
        WHEN gameDate BETWEEN '1979-10-12' AND '1980-05-16' THEN 1979
        WHEN gameDate BETWEEN '1978-10-13' AND '1979-06-01' THEN 1978
        WHEN gameDate BETWEEN '1977-10-18' AND '1978-06-07' THEN 1977
        WHEN gameDate BETWEEN '1976-02-13' AND '1977-06-05' THEN 1976
        WHEN gameDate BETWEEN '1976-02-03' AND '1976-06-06' THEN 1975
        WHEN gameDate BETWEEN '1974-10-17' AND '1975-05-25' THEN 1974
        WHEN gameDate BETWEEN '1973-10-09' AND '1974-05-12' THEN 1973
        WHEN gameDate BETWEEN '1972-10-10' AND '1973-05-10' THEN 1972
        WHEN gameDate BETWEEN '1971-10-12' AND '1972-05-07' THEN 1971
        WHEN gameDate BETWEEN '1971-01-12' AND '1971-04-30' THEN 1970
        WHEN gameDate BETWEEN '1969-10-14' AND '1970-05-08' THEN 1969
        WHEN gameDate BETWEEN '1968-10-15' AND '1969-03-24' THEN 1968
        WHEN gameDate BETWEEN '1967-10-13' AND '1968-05-02' THEN 1967
        WHEN gameDate BETWEEN '1967-01-10' AND '1967-04-24' THEN 1966
        WHEN gameDate BETWEEN '1965-10-15' AND '1966-04-28' THEN 1965
        WHEN gameDate BETWEEN '1964-10-16' AND '1965-03-21' THEN 1964
        WHEN gameDate BETWEEN '1963-10-16' AND '1964-04-26' THEN 1963
        WHEN gameDate BETWEEN '1962-10-19' AND '1963-04-24' THEN 1962
        WHEN gameDate BETWEEN '1961-02-16' AND '1962-04-18' THEN 1961
        WHEN gameDate BETWEEN '1961-01-17' AND '1961-01-17' THEN 1960
        WHEN gameDate BETWEEN '1959-10-18' AND '1960-04-09' THEN 1959
        WHEN gameDate BETWEEN '1958-10-19' AND '1959-03-11' THEN 1958
        WHEN gameDate BETWEEN '1957-10-22' AND '1958-03-12' THEN 1957
        WHEN gameDate BETWEEN '1956-10-27' AND '1957-04-13' THEN 1956
        WHEN gameDate BETWEEN '1955-11-05' AND '1956-04-05' THEN 1955
        WHEN gameDate BETWEEN '1954-10-30' AND '1955-04-10' THEN 1954
        WHEN gameDate BETWEEN '1953-10-30' AND '1954-04-11' THEN 1953
        WHEN gameDate BETWEEN '1952-10-31' AND '1953-04-10' THEN 1952
        WHEN gameDate BETWEEN '1951-11-01' AND '1952-04-23' THEN 1951
        WHEN gameDate BETWEEN '1950-10-31' AND '1951-04-21' THEN 1950
        WHEN gameDate BETWEEN '1949-10-29' AND '1950-04-23' THEN 1949
        WHEN gameDate BETWEEN '1948-11-01' AND '1949-04-13' THEN 1948
        WHEN gameDate BETWEEN '1947-11-12' AND '1948-04-21' THEN 1947
        WHEN gameDate BETWEEN '1946-11-01' AND '1947-04-22' THEN 1946
    END AS season
FROM player_statistics_test_df


"""

player_statistics_test_df = duckdb.query(query).df()
player_statistics_test_df

,firstName,lastName,full_name,player_id,gameId,gameDate,playerteamCity,playerteamName,opponentteamCity,opponentteamName,...,freeThrowsAttempted,freeThrowsMade,freeThrowsPercentage,reboundsDefensive,reboundsOffensive,reboundsTotal,foulsPersonal,turnovers,plusMinusPoints,season
0,Devin,Booker,Devin Booker,1626164,22500239,2025-11-16T20:00:00Z,Phoenix,Suns,Atlanta,Hawks,...,7.0,7.0,1.000,1.0,1.0,2.0,4.0,5.0,-17.0,2025
1,Royce,O'Neale,Royce O'Neale,1626220,22500239,2025-11-16T20:00:00Z,Phoenix,Suns,Atlanta,Hawks,...,0.0,0.0,0.000,4.0,2.0,6.0,1.0,0.0,-18.0,2025
2,Lauri,Markkanen,Lauri Markkanen,1628374,22500240,2025-11-16T20:00:00Z,Utah,Jazz,Chicago,Bulls,...,13.0,11.0,0.846,5.0,2.0,7.0,3.0,1.0,4.0,2025
3,Luke,Kennard,Luke Kennard,1628379,22500239,2025-11-16T20:00:00Z,Atlanta,Hawks,Phoenix,Suns,...,0.0,0.0,0.000,1.0,0.0,1.0,0.0,0.0,-4.0,2025
4,Dillon,Brooks,Dillon Brooks,1628415,22500239,2025-11-16T20:00:00Z,Phoenix,Suns,Atlanta,Hawks,...,3.0,3.0,1.000,5.0,1.0,6.0,5.0,3.0,-2.0,2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1635253,Fred,Sheffield,Fred Sheffield,78131,24600052,1946-11-26 19:00:00,Philadelphia,Warriors,Boston,Celtics,...,0.0,1.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,1946
1635254,Connie,Simmons,Connie Simmons,78153,24600052,1946-11-26 19:00:00,Boston,Celtics,Philadelphia,Warriors,...,0.0,5.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,1946
1635255,Johnny,Simmons,Johnny Simmons,78154,24600052,1946-11-26 19:00:00,Boston,Celtics,Philadelphia,Warriors,...,0.0,2.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,1946
1635256,Virgil,Vaughn,Virgil Vaughn,78411,24600052,1946-11-26 19:00:00,Boston,Celtics,Philadelphia,Warriors,...,0.0,2.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,1946


In [40]:
nba_all_stars_df = pd.read_csv('nba-all-stars.csv')
nba_all_stars_df["Season"] = nba_all_stars_df["Selection Year"] - 1
nba_all_stars_df["player_id"] = nba_all_stars_df["player_id"].astype(int)
nba_all_stars_df

,player_id,player_name,Total Selections,Selection Year,Season
0,2544,LeBron James,21,2005,2004
1,2544,LeBron James,21,2006,2005
2,2544,LeBron James,21,2007,2006
3,2544,LeBron James,21,2008,2007
4,2544,LeBron James,21,2009,2008
...,...,...,...,...,...
1816,913,Larry D. Johnson,2,1995,1994
1817,77376,Lafayette Lever,2,1988,1987
1818,77376,Lafayette Lever,2,1990,1989
1819,76753,World Free,1,1980,1979


In [41]:
nba_mvp_df = pd.read_csv('nba-mvp.csv')
nba_mvp_df["Season"] = nba_mvp_df["Season"].str[:4].astype(int)
nba_mvp_df

,player_id,player_name,Season,Team,Pos,Height,Weight,Age,Pre-Draft Team,Draft Yr,YOS
0,1628983,Shai Gilgeous-Alexander,2024,Oklahoma City Thunder,PG,6-6,195,26,Kentucky,2018,6
1,203999,Nikola Jokic,2023,Denver Nuggets,C,6-11,284,29,KK Mega Bemax (Serbia),2014,8
2,203954,Joel Embiid,2022,Philadelphia Sixers,C,7-0,280,28,Kansas,2014,8
3,203999,Nikola Jokic,2021,Denver Nuggets,C,6-11,284,27,KK Mega Bemax (Serbia),2014,6
4,203999,Nikola Jokic,2020,Denver Nuggets,C,6-11,284,26,KK Mega Bemax (Serbia),2014,5
...,...,...,...,...,...,...,...,...,...,...,...
65,76375,Wilt Chamberlain,1959,San Francisco Warriors,C,7-1,265,23,Kansas,1959,0
66,77847,Bob Pettit,1958,St. Louis Hawks,FC,6-9,205,26,LSU,1954,4
67,78049,Bill Russell,1957,Boston Celtics,C,6-10,215,24,San Francisco,1956,1
68,600003,Bob Cousy,1956,Boston Celtics,PG,6-1,175,28,Holy Cross,1950,6


In [42]:
all_nba_first_team_df = pd.read_csv('all-nba-first-team.csv')
all_nba_first_team_df["Season"] = all_nba_first_team_df["Season"].str[:4].astype(int)
all_nba_first_team_df

all_nba_second_team_df = pd.read_csv('all-nba-second-team.csv')
all_nba_second_team_df["Season"] = all_nba_second_team_df["Season"].str[:4].astype(int)
all_nba_second_team_df

all_nba_third_team_df = pd.read_csv('all-nba-third-team.csv')
all_nba_third_team_df["Season"] = all_nba_third_team_df["Season"].str[:4].astype(int)
all_nba_third_team_df

,player_id,player_name,Season,Team,Pos,Height,Weight,Age,Pre-Draft Team,Draft Yr,YOS
0,1630595,Cade Cunningham,2024,Detroit Pistons,SF,6-6,220,23,Oklahoma State,2021,3
1,1630169,Tyrese Haliburton,2024,Indiana Pacers,PG,6-5,185,25,Iowa State,2020,4
2,1631114,Jalen Williams,2024,Oklahoma City Thunder,G,6-5,211,23,Santa Clara,2022,2
3,201935,James Harden,2024,Los Angeles Clippers,SG,6-5,220,35,Arizona State,2009,15
4,1626157,Karl-Anthony Towns,2024,New York Knicks,C,7-0,248,29,Kentucky,2015,9
...,...,...,...,...,...,...,...,...,...,...,...
180,305,Robert Parish,1988,Boston Celtics,C,7-0,230,35,Centenary (LA),1976,11
181,1122,Dominique Wilkins,1988,Atlanta Hawks,SF,6-7,200,29,Georgia,1982,6
182,107,Dale Ellis,1988,Seattle SuperSonics,GF,6-7,205,28,Tennessee,1983,5
183,899,Mark Price,1988,Cleveland Cavaliers,PG,6-0,170,25,Georgia Tech,1986,2


In [45]:
query = """

WITH All_Star_Flag AS
(
SELECT *,
'Yes' AS all_star_this_season
FROM nba_all_stars_df
)

,MVP_Flag AS (
SELECT *,
'Yes' AS mvp_this_season
FROM nba_mvp_df
)

,All_NBA_First_Team_Flag AS (
SELECT *,
'Yes' AS all_nba_first_team_this_season
FROM all_nba_first_team_df
)

,All_NBA_Second_Team_Flag AS (
SELECT *,
'Yes' AS all_nba_second_team_this_season
FROM all_nba_second_team_df
)

,All_NBA_Third_Team_Flag AS (
SELECT *,
'Yes' AS all_nba_third_team_this_season
FROM all_nba_third_team_df
)

SELECT
player_statistics_test_df.firstName
,player_statistics_test_df.lastName
,player_statistics_test_df.full_name
,player_statistics_test_df.player_id
,player_statistics_test_df.gameId
,CAST(player_statistics_test_df.gameDate AS DATE) AS gameDate
,player_statistics_test_df.playerteamCity
,player_statistics_test_df.playerteamName
,player_statistics_test_df.opponentteamCity
,player_statistics_test_df.opponentteamName
,player_statistics_test_df.gameType
,player_statistics_test_df.gameLabel
,player_statistics_test_df.gameSubLabel
,player_statistics_test_df.seriesGameNumber
,player_statistics_test_df.win
,player_statistics_test_df.home
,player_statistics_test_df.numMinutes
,CAST(player_statistics_test_df.points AS INT) AS points
,CAST(player_statistics_test_df.assists AS INT) AS assists
,CAST(player_statistics_test_df.blocks AS INT) AS blocks
,CAST(player_statistics_test_df.steals AS INT) AS steals
,CAST(player_statistics_test_df.fieldGoalsAttempted AS INT) AS fieldGoalsAttempted
,CAST(player_statistics_test_df.fieldGoalsMade AS INT) AS fieldGoalsMade
,player_statistics_test_df.fieldGoalsPercentage
,CAST(player_statistics_test_df.threePointersAttempted AS INT) AS threePointersAttempted
,CAST(player_statistics_test_df.threePointersMade AS INT) AS threePointersMade
,player_statistics_test_df.threePointersPercentage
,CAST(player_statistics_test_df.freeThrowsAttempted AS INT) AS freeThrowsAttempted
,CAST(player_statistics_test_df.freeThrowsMade AS INT) AS freeThrowsMade
,player_statistics_test_df.freeThrowsPercentage
,CAST(player_statistics_test_df.reboundsDefensive AS INT) AS reboundsDefensive
,CAST(player_statistics_test_df.reboundsOffensive AS INT) AS reboundsOffensive
,CAST(player_statistics_test_df.reboundsTotal AS INT) AS reboundsTotal
,CAST(player_statistics_test_df.foulsPersonal AS INT) AS foulsPersonal
,CAST(player_statistics_test_df.turnovers AS INT) AS turnovers
,CAST(player_statistics_test_df.plusMinusPoints AS INT) AS plusMinusPoints
,player_statistics_test_df.season

,All_Star_Flag.all_star_this_season
,MVP_Flag.mvp_this_season
,All_NBA_First_Team_Flag.all_nba_first_team_this_season
,All_NBA_Second_Team_Flag.all_nba_second_team_this_season
,All_NBA_Third_Team_Flag.all_nba_third_team_this_season

FROM player_statistics_test_df
LEFT JOIN All_Star_Flag
ON All_Star_Flag.player_id = player_statistics_test_df.player_id AND All_Star_Flag.Season = player_statistics_test_df.Season
LEFT JOIN MVP_Flag
ON MVP_Flag.player_id = player_statistics_test_df.player_id AND MVP_Flag.Season = player_statistics_test_df.Season
LEFT JOIN All_NBA_First_Team_Flag
ON All_NBA_First_Team_Flag.player_id = player_statistics_test_df.player_id AND All_NBA_First_Team_Flag.Season = player_statistics_test_df.Season
LEFT JOIN All_NBA_Second_Team_Flag
ON All_NBA_Second_Team_Flag.player_id = player_statistics_test_df.player_id AND All_NBA_Second_Team_Flag.Season = player_statistics_test_df.Season
LEFT JOIN All_NBA_Third_Team_Flag
ON All_NBA_Third_Team_Flag.player_id = player_statistics_test_df.player_id AND All_NBA_Third_Team_Flag.Season = player_statistics_test_df.Season


"""

result_df = duckdb.query(query).df()
result_df["all_star_this_season"] = np.where(result_df["all_star_this_season"] == 'Yes',1,0)
result_df["mvp_this_season"] = np.where(result_df["mvp_this_season"] == 'Yes',1,0)
result_df["all_nba_first_team_this_season"] = np.where(result_df["all_nba_first_team_this_season"] == 'Yes',1,0)
result_df["all_nba_second_team_this_season"] = np.where(result_df["all_nba_second_team_this_season"] == 'Yes',1,0)
result_df["all_nba_third_team_this_season"] = np.where(result_df["all_nba_third_team_this_season"] == 'Yes',1,0)
player_stats_with_allstar_mvp_allnba_df = result_df
player_stats_with_allstar_mvp_allnba_df

,firstName,lastName,full_name,player_id,gameId,gameDate,playerteamCity,playerteamName,opponentteamCity,opponentteamName,...,reboundsTotal,foulsPersonal,turnovers,plusMinusPoints,season,all_star_this_season,mvp_this_season,all_nba_first_team_this_season,all_nba_second_team_this_season,all_nba_third_team_this_season
0,Seth,Curry,Seth Curry,203552,21900559,2020-01-08,Dallas,Mavericks,Denver,Nuggets,...,1,1,0,-5,2019,0,0,0,0,0
1,Gary,Harris,Gary Harris,203914,21900559,2020-01-08,Denver,Nuggets,Dallas,Mavericks,...,0,2,1,-7,2019,0,0,0,0,0
2,Jerami,Grant,Jerami Grant,203924,21900559,2020-01-08,Denver,Nuggets,Dallas,Mavericks,...,6,4,0,-5,2019,0,0,0,0,0
3,Dwight,Powell,Dwight Powell,203939,21900559,2020-01-08,Dallas,Mavericks,Denver,Nuggets,...,8,4,2,6,2019,0,0,0,0,0
4,Clint,Capela,Clint Capela,203991,21900558,2020-01-08,Houston,Rockets,Atlanta,Hawks,...,22,1,1,-10,2019,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1635253,Paul,Pierce,Paul Pierce,1718,20200018,2002-10-31,Boston,Celtics,Washington,Wizards,...,6,1,1,-23,2002,1,0,0,0,1
1635254,Jamal,Mashburn,Jamal Mashburn,469,20200013,2002-10-30,New Orleans,Hornets,Utah,Jazz,...,9,2,3,8,2002,1,0,0,0,1
1635255,Steve,Nash,Steve Nash,959,20200012,2002-10-30,Dallas,Mavericks,Memphis,Grizzlies,...,1,4,3,12,2002,1,0,0,0,1
1635256,Paul,Pierce,Paul Pierce,1718,20200004,2002-10-30,Boston,Celtics,Chicago,Bulls,...,7,4,6,2,2002,1,0,0,0,1


In [67]:
query = """
WITH Overall_Data AS(
SELECT player_stats_with_pow.*
,player_stats_with_allstar_mvp_allnba_df.season
,all_star_this_season
,mvp_this_season
,all_nba_first_team_this_season
,all_nba_second_team_this_season
,all_nba_third_team_this_season
FROM
player_stats_with_pow
JOIN player_stats_with_allstar_mvp_allnba_df
ON (player_stats_with_pow.player_id = player_stats_with_allstar_mvp_allnba_df.player_id AND player_stats_with_pow.gameId = player_stats_with_allstar_mvp_allnba_df.gameId)
)

SELECT

firstName
,lastName
,full_name
,player_id
,gameId
,gameDate

,team
,teamid
,opponent
,opponentid
,outcome
,home
,team_score
,opp_score
,games_prior
,wins_prior
,losses_prior
,record_prior
,home_games_prior
,home_wins_prior
,home_losses_prior
,home_record_prior
,away_games_prior
,away_wins_prior
,away_losses_prior
,away_record_prior
,win_streak_prior
,home_win_streak_prior
,away_win_streak_prior
,opp_wins_prior
,opp_losses_prior
,opp_winrate_prior
,is_win_vs_over_500
,wins_vs_over_500_prior
,week_games_prior
,week_wins_prior
,week_losses_prior
,week_record_prior
,conference


,season
,numMinutes
,CAST(points AS INT) AS points
,CAST(assists AS INT) AS assists
,CAST(blocks AS INT) AS blocks
,CAST(steals AS INT) AS steals
,CAST(fieldGoalsAttempted AS INT) AS fieldGoalsAttempted
,CAST(fieldGoalsMade AS INT) AS fieldGoalsMade
,fieldGoalsPercentage
,CAST(threePointersAttempted AS INT) AS threePointersAttempted
,CAST(threePointersMade AS INT) AS threePointersMade
,threePointersPercentage
,CAST(freeThrowsAttempted AS INT) AS freeThrowsAttempted
,CAST(freeThrowsMade AS INT) AS freeThrowsMade
,freeThrowsPercentage
,CAST(reboundsDefensive AS INT) AS reboundsDefensive
,CAST(reboundsOffensive AS INT) AS reboundsOffensive
,CAST(reboundsTotal AS INT) AS reboundsTotal
,CAST(foulsPersonal AS INT) AS foulsPersonal
,CAST(turnovers AS INT) AS turnovers
,CAST(plusMinusPoints AS INT) AS plusMinusPoints
,won_player_of_the_week
,all_star_this_season
,mvp_this_season
,all_nba_first_team_this_season
,all_nba_second_team_this_season
,all_nba_third_team_this_season

FROM Overall_Data
"""

overall_df = duckdb.query(query).df()
overall_df

,firstName,lastName,full_name,player_id,gameId,gameDate,team,teamid,opponent,opponentid,...,reboundsTotal,foulsPersonal,turnovers,plusMinusPoints,won_player_of_the_week,all_star_this_season,mvp_this_season,all_nba_first_team_this_season,all_nba_second_team_this_season,all_nba_third_team_this_season
0,Chris,Mihm,Chris Mihm,2036,20000986,2001-03-22,None,<NA>,None,<NA>,...,7,4,1,5,0,0,0,0,0,0
1,Jerome,Moiso,Jerome Moiso,2040,20000984,2001-03-22,None,<NA>,None,<NA>,...,0,0,1,0,0,0,0,0,0,0
2,Soumaila,Samake,Soumaila Samake,2066,20000984,2001-03-22,None,<NA>,None,<NA>,...,1,0,1,4,0,0,0,0,0,0
3,Hanno,Mottola,Hanno Mottola,2069,20000985,2001-03-22,None,<NA>,None,<NA>,...,3,3,0,-8,0,0,0,0,0,0
4,Clar.,Weatherspoon,Clar. Weatherspoon,221,20000986,2001-03-22,None,<NA>,None,<NA>,...,13,1,0,24,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1476968,Oscar,Robertson,Oscar Robertson,600015,26900525,1970-03-12,None,<NA>,None,<NA>,...,12,1,0,0,0,1,0,0,0,0
1476969,Wally,Anderzunas,Wally Anderzunas,76048,26900525,1970-03-12,None,<NA>,None,<NA>,...,3,3,0,0,0,0,0,0,0,0
1476970,Jim,Barnett,Jim Barnett,76106,26900525,1970-03-12,None,<NA>,None,<NA>,...,0,2,0,0,0,0,0,0,0,0
1476971,John,Block,John Block,76187,26900525,1970-03-12,None,<NA>,None,<NA>,...,9,4,0,0,0,0,0,0,0,0


In [68]:
overall_df.columns

Index(['firstName', 'lastName', 'full_name', 'player_id', 'gameId', 'gameDate',
       'team', 'teamid', 'opponent', 'opponentid', 'outcome', 'home',
       'team_score', 'opp_score', 'games_prior', 'wins_prior', 'losses_prior',
       'record_prior', 'home_games_prior', 'home_wins_prior',
       'home_losses_prior', 'home_record_prior', 'away_games_prior',
       'away_wins_prior', 'away_losses_prior', 'away_record_prior',
       'win_streak_prior', 'home_win_streak_prior', 'away_win_streak_prior',
       'opp_wins_prior', 'opp_losses_prior', 'opp_winrate_prior',
       'is_win_vs_over_500', 'wins_vs_over_500_prior', 'week_games_prior',
       'week_wins_prior', 'week_losses_prior', 'week_record_prior',
       'conference', 'season', 'numMinutes', 'points', 'assists', 'blocks',
       'steals', 'fieldGoalsAttempted', 'fieldGoalsMade',
       'fieldGoalsPercentage', 'threePointersAttempted', 'threePointersMade',
       'threePointersPercentage', 'freeThrowsAttempted', 'freeThrowsMade',

In [69]:
overall_df.columns[overall_df.isnull().any()]

Index(['team', 'teamid', 'opponent', 'opponentid', 'outcome', 'home',
       'team_score', 'opp_score', 'games_prior', 'wins_prior', 'losses_prior',
       'record_prior', 'home_games_prior', 'home_wins_prior',
       'home_losses_prior', 'home_record_prior', 'away_games_prior',
       'away_wins_prior', 'away_losses_prior', 'away_record_prior',
       'win_streak_prior', 'home_win_streak_prior', 'away_win_streak_prior',
       'opp_wins_prior', 'opp_losses_prior', 'opp_winrate_prior',
       'is_win_vs_over_500', 'wins_vs_over_500_prior', 'week_games_prior',
       'week_wins_prior', 'week_losses_prior', 'week_record_prior',
       'conference', 'season'],
      dtype='object')

In [74]:
overall_df["home_games_prior"].head(50)

,home_games_prior
0,<NA>
1,<NA>
2,<NA>
3,<NA>
4,<NA>
5,<NA>
6,<NA>
7,<NA>
8,<NA>
9,<NA>
